## Automated trading for dual moving average crossover strategy on Single Asset using Interactive Brokers API

Reference: https://github.com/erdewit/ib_insync/blob/master/notebooks/

Remarks: haven't done yet

Prerequisites:
1. Install TWS API: https://interactivebrokers.github.io/#
2. `pip install ib_insync`

In [431]:
import yfinance as yf
from ib_insync import *

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)

In [420]:
asset_to_trade = "BTC"
yahoo_stock_code = "BTC-USD"

market = "PAXOS" # "SMART"

currency = "USD"
cashQty = 1e4

In [421]:
ib = IB()
ib.connect("127.0.0.1", 7497, clientId=2) # replace with your connection details

<IB connected to 127.0.0.1:7497 clientId=2>

In [422]:
ib.positions()

[]

In [423]:
[v for v in ib.accountValues() if v.tag == 
 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU6799464', tag='NetLiquidationByCurrency', value='1022025.47', currency='BASE', modelCode='')]

In [424]:
# define the asset we're interested in
contract = Crypto(asset_to_trade, market, currency)
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='CRYPTO', conId=479624278, symbol='BTC', exchange='PAXOS', currency='USD', localSymbol='BTC.USD', tradingClass='BTC'), marketName='BTC', minTick=0.25, orderTypes='AD,ADJUST,ALERT,ALLOC,BASKET,CASHQTY,COND,DAY,DEACT,DEACTDIS,DEACTEOD,GAT,GTC,GTD,GTT,IOC,LMT,MKT,NMIN,NONALGO,WHATIF', validExchanges='PAXOS', priceMagnifier=1, underConId=0, longName='Bitcoin cryptocurrency', contractMonth='', industry='', category='', subcategory='', timeZoneId='US/Eastern', tradingHours='20230812:CLOSED;20230813:0300-20230814:1600;20230814:1601-20230815:1600;20230815:1601-20230816:1600;20230816:1601-20230817:1600;20230817:1601-20230818:1600', liquidHours='20230812:CLOSED;20230813:0300-20230814:1600;20230814:1601-20230815:1600;20230815:1601-20230816:1600;20230816:1601-20230817:1600;20230817:1601-20230818:1600', evRule='', evMultiplier=0, mdSizeMultiplier=1, aggGroup=2147483647, underSymbol='', underSecType='', marketRuleIds='2806', secIdList=[], realExpirationDate

In [425]:
# fetch data from yahoo finance
data= yf.download(yahoo_stock_code,start="2020-01-01",end="2022-12-31")


[*********************100%***********************]  1 of 1 completed


In [426]:
# Calculate moving average
data["MA7"] = data["Close"].rolling(window=7).mean()
data["MA14"] = data["Close"].rolling(window=14).mean()

In [427]:
# Create signals
data["Buy_Signal"] = data["MA7"] > data["MA14"]
data["Sell_Signal"] = data["MA7"] < data["MA14"]

In [428]:
#exchange_hours = ib.reqMarketDataType(contract)
#exchange_hours # cannot works and suspected need to buy the data

In [429]:
# check for buy signal
if data["Buy_Signal"].iloc[-1]:
    order = LimitOrder("SELL", 1, lmtPrice= data["MA7"].iloc[-1], tif="GTC") # good-til-canceled
    trade = ib.placeOrder(contract, order)
    ib.sleep(1) # Wait a second for trade to execute
    print("SELL")

# check for sell signal
elif data["Sell_Signal"].iloc[-1]:
    order = LimitOrder("BUY",1, lmtPrice = data["MA14"].iloc[-1], tif="GTC") # good-til-canceled
    trade = ib.placeOrder(contract, order)
    ib.sleep(1)
    print("BUY")
    

BUY


In [430]:
ib.disconnect()

Peer closed connection.
Peer closed connection.
